# Project Description
In Module 3, we explored New York City and the city of Toronto and segmented and clustered their neighborhoods. Both cities are very diverse and are the financial capitals of their respective countries. One of the ideas would be to compare the neighborhoods of the two cities and determine how similar or dissimilar they are.# Part 1: Dataframe: PostalCode, Borough, Neighborhood



# Target Audience
Target audience would be investors or future restuarants who want to determine the best place to place their investments of to build their next restaurant

# Data
Used the Toronto data to find top venues around each address and area, allowing users and businesses to find an optimal area for investment of building of restauraunts.

Also used location data from Foursquare to determine best venues for each locations and rank them.

# Methodology
used K means clustering to find clusters based on patterns from the venue data

# Results
based on the clustering and data, restauraunts and cafes are among the more popular venues, 

In [7]:
import pandas as pd
import numpy as np
import json
from bs4 import BeautifulSoup
import requests
import pandas

In [8]:
soup = BeautifulSoup(requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text, 'lxml')
table = soup.find('table',{'class':'wikitable sortable'})
table_rows = table.find_all('tr')
data = []
for row in table_rows:
    data.append([t.text.strip() for t in row.find_all('td')])

df = pandas.DataFrame(data, columns=['PostalCode', 'Borough', 'Neighbourhood'])
df = df[~df['PostalCode'].isnull()]

In [9]:
df.head()

,PostalCode,Borough,Neighbourhood
1,M1A,Not assigned,Not assigned
2,M2A,Not assigned,Not assigned
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [10]:
df.drop(df[df['Borough']=="Not assigned"].index,axis=0, inplace=True)

In [11]:
df.groupby('PostalCode').agg(lambda x: ','.join(x))

,Borough,Neighbourhood
PostalCode,,
M1B,Scarborough,"Malvern, Rouge"
M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
M1E,Scarborough,"Guildwood, Morningside, West Hill"
M1G,Scarborough,Woburn
M1H,Scarborough,Cedarbrae
...,...,...
M9N,York,Weston
M9P,Etobicoke,Westmount
M9R,Etobicoke,"Kingsview Village, St. Phillips, Martin Grove ..."


In [12]:
df.loc[df['Neighbourhood']=="Not assigned",'Neighbourhood']=df.loc[df['Neighbourhood']=="Not assigned",'Borough']

In [13]:
df['Borough']= df['Borough'].str.replace('nan|[{}\s]','').str.split(',').apply(set).str.join(',').str.strip(',').str.replace(",{2,}",",")

In [14]:
df.reset_index()

,index,PostalCode,Borough,Neighbourhood
0,3,M3A,NorthYork,Parkwoods
1,4,M4A,NorthYork,Victoria Village
2,5,M5A,DowntownToronto,"Regent Park, Harbourfront"
3,6,M6A,NorthYork,"Lawrence Manor, Lawrence Heights"
4,7,M7A,DowntownToronto,"Queen's Park, Ontario Provincial Government"
...,...,...,...,...
98,161,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
99,166,M4Y,DowntownToronto,Church and Wellesley
100,169,M7Y,EastToronto,"Business reply mail Processing Centre, South C..."
101,170,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


In [15]:
df.shape

(103, 3)

In [16]:
pip install geopy


Note: you may need to restart the kernel to use updated packages.


In [17]:
from  geopy.geocoders import Nominatim

In [18]:
df_geo = df

In [19]:
df_geo['address'] = df[['PostalCode', 'Borough', 'Neighbourhood']].apply(lambda x: ', '.join(x), axis=1 )

In [20]:
df_geo.head()
df_geo.shape

(103, 4)

In [21]:
!conda install -c conda-forge geocoder --yes
print("Installation Done!")
import geocoder # import geocoder
print("Geo Coder imported!")

Solving environment: done

# All requested packages already installed.

Installation Done!
Geo Coder imported!


In [22]:
def get_geocoder(postal_code_from_df):
    # initialize your variable to None
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Toronto, Ontario'.format(postal_code_from_df.strip()))
        lat_lng_coords = g.latlng
        latitude = lat_lng_coords[0]
        longitude = lat_lng_coords[1]
    return latitude,longitude

In [23]:
df['Latitude'], df['Longitude'] = zip(*df['PostalCode'].apply(get_geocoder))
df.head()

,PostalCode,Borough,Neighbourhood,address,Latitude,Longitude
3,M3A,NorthYork,Parkwoods,"M3A, NorthYork, Parkwoods",43.75188,-79.33036
4,M4A,NorthYork,Victoria Village,"M4A, NorthYork, Victoria Village",43.73042,-79.31282
5,M5A,DowntownToronto,"Regent Park, Harbourfront","M5A, DowntownToronto, Regent Park, Harbourfront",43.65514,-79.36265
6,M6A,NorthYork,"Lawrence Manor, Lawrence Heights","M6A, NorthYork, Lawrence Manor, Lawrence Heights",43.72321,-79.45141
7,M7A,DowntownToronto,"Queen's Park, Ontario Provincial Government","M7A, DowntownToronto, Queen's Park, Ontario Pr...",43.66449,-79.39302


In [24]:
geolocator = Nominatim(user_agent="toronto_ontario")
location = geolocator.geocode('Toronto, Ontario')
latitude = location.latitude
longitude = location.longitude

In [27]:
import folium
torontomap = folium.Map(location=[latitude, longitude], zoom_start=11)

for lat, long, post, borough, neigh in zip(df['Latitude'], df['Longitude'], df['PostalCode'], df['Borough'], df['Neighbourhood']):
    label = "{} ({}): {}".format(borough, post, neigh)
    popup = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, long],
        radius=5,
        popup=popup,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(torontomap)
    
torontomap

In [28]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Postcode Latitude', 
                  'Postcode Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [29]:
boroughs = set(df['Borough'])
toronto_boroughs = {borough for borough in boroughs if 'Toronto' in borough}
toronto_data = df[df['Borough'].isin(toronto_boroughs)].reset_index(drop=True)
toronto_data.head()

,PostalCode,Borough,Neighbourhood,address,Latitude,Longitude
0,M5A,DowntownToronto,"Regent Park, Harbourfront","M5A, DowntownToronto, Regent Park, Harbourfront",43.65514,-79.36265
1,M7A,DowntownToronto,"Queen's Park, Ontario Provincial Government","M7A, DowntownToronto, Queen's Park, Ontario Pr...",43.66449,-79.39302
2,M5B,DowntownToronto,"Garden District, Ryerson","M5B, DowntownToronto, Garden District, Ryerson",43.65736,-79.37818
3,M5C,DowntownToronto,St. James Town,"M5C, DowntownToronto, St. James Town",43.65143,-79.37557
4,M4E,EastToronto,The Beaches,"M4E, EastToronto, The Beaches",43.67703,-79.29542


In [30]:
CLIENT_ID = '414WGKUT1A5VY5DNDWHFRSQYSK3OSOXHOGFH0HULYPHZ55L3' # your Foursquare ID
CLIENT_SECRET = 'XOCNT3ETC5L4W0NFJABMYFQRRDA23JP5DTVTNTWNJSFDAQE3' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

In [32]:
toronto_venues = getNearbyVenues(toronto_data.Neighbourhood, toronto_data.Latitude, toronto_data.Longitude)
toronto_venues.head(5)

,Neighborhood,Postcode Latitude,Postcode Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park, Harbourfront",43.65514,-79.36265,Roselle Desserts,43.653447,-79.362017,Bakery
1,"Regent Park, Harbourfront",43.65514,-79.36265,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,"Regent Park, Harbourfront",43.65514,-79.36265,Figs Breakfast & Lunch,43.655675,-79.364503,Breakfast Spot
3,"Regent Park, Harbourfront",43.65514,-79.36265,The Yoga Lounge,43.655515,-79.364955,Yoga Studio
4,"Regent Park, Harbourfront",43.65514,-79.36265,Body Blitz Spa East,43.654735,-79.359874,Spa


In [33]:
toronto_venues.groupby('Neighborhood').count()


,Postcode Latitude,Postcode Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,30,30,30,30,30,30
"Brockton, Parkdale Village, Exhibition Place",30,30,30,30,30,30
"Business reply mail Processing Centre, South Central Letter Processing Plant Toronto",30,30,30,30,30,30
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",30,30,30,30,30,30
Central Bay Street,30,30,30,30,30,30
Christie,10,10,10,10,10,10
Church and Wellesley,30,30,30,30,30,30
"Commerce Court, Victoria Hotel",30,30,30,30,30,30
Davisville,27,27,27,27,27,27


In [41]:
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped.head(5)

,Neighborhood,Yoga Studio,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,BBQ Joint,...,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop
0,Berczy Park,0.000000,0.0,0.0,0.0,0.033333,0.000000,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.033333,0.0,0.0,0.0,0.0
1,"Brockton, Parkdale Village, Exhibition Place",0.000000,0.0,0.0,0.0,0.000000,0.000000,0.033333,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.033333,0.0,0.0,0.0,0.0
2,"Business reply mail Processing Centre, South C...",0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.033333,0.0,0.0,0.0,0.0
3,"CN Tower, King and Spadina, Railway Lands, Har...",0.033333,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.033333,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
4,Central Bay Street,0.000000,0.0,0.0,0.0,0.000000,0.033333,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0


In [42]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [43]:
num_top_venues = 10
indicators = ['st', 'nd', 'rd']

columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] =toronto_grouped['Neighborhood']

def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head(5)

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Beer Bar,Cocktail Bar,Seafood Restaurant,Tailor Shop,Café,Farmers Market,Restaurant,Jazz Club,Breakfast Spot,Coffee Shop
1,"Brockton, Parkdale Village, Exhibition Place",Gift Shop,Café,Furniture / Home Store,Restaurant,Bakery,Coffee Shop,Supermarket,Seafood Restaurant,Sandwich Place,Breakfast Spot
2,"Business reply mail Processing Centre, South C...",Café,Coffee Shop,Restaurant,Concert Hall,Theater,Hotel,Steakhouse,Opera House,Japanese Restaurant,Brazilian Restaurant
3,"CN Tower, King and Spadina, Railway Lands, Har...",Italian Restaurant,Gym / Fitness Center,Coffee Shop,Yoga Studio,Fish & Chips Shop,Brewery,Burger Joint,Restaurant,Café,Ramen Restaurant
4,Central Bay Street,Coffee Shop,Plaza,Japanese Restaurant,Clothing Store,Poke Place,Shopping Mall,Italian Restaurant,Modern European Restaurant,Bubble Tea Shop,Ramen Restaurant


In [53]:
from sklearn.cluster import KMeans
toronto_grouped_clustering = toronto_grouped.drop('Neighbourhood', 1)


KeyError: "['Neighbourhood'] not found in axis"

In [55]:
import matplotlib.cm as cm
import matplotlib.colors as colors
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=12)

# set color scheme for the clusters
x = np.arange(5)
ys = [i+x+(i*x)**2 for i in range(5)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighbourhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters